##### Day 3 — SQL schema

###### Step 1 — Import dependencies & load CSV

In [9]:
import pandas as pd
import sqlite3

# Step 1 — Load CSV
df = pd.read_csv(r"C:/Users/HP/supply-chain-delay/data/raw/smart_logistics_dataset.csv")

# Preview the dataset
print(df.head())
print("\nColumns in dataset:", df.columns.tolist())


             Timestamp  Asset_ID  Latitude  Longitude  Inventory_Level  \
0  2024-03-20 00:11:14   Truck_7  -65.7383    11.2497              390   
1  2024-10-30 07:53:51   Truck_6   22.2748  -131.7086              491   
2  2024-07-29 18:42:48  Truck_10   54.9232    79.5455              190   
3  2024-10-28 00:50:54   Truck_9   42.3900    -1.4788              330   
4  2024-09-27 15:52:58   Truck_7  -65.8477    47.9468              480   

  Shipment_Status  Temperature  Humidity Traffic_Status  Waiting_Time  \
0         Delayed         27.0      67.8         Detour            38   
1      In Transit         22.5      54.3          Heavy            16   
2      In Transit         25.2      62.2         Detour            34   
3       Delivered         25.4      52.3          Heavy            37   
4         Delayed         20.5      57.2          Clear            56   

   User_Transaction_Amount  User_Purchase_Frequency Logistics_Delay_Reason  \
0                      320            

###### Step 2 — Connect to SQLite DB

In [15]:
# Create SQLite DB file
conn = sqlite3.connect("smart_logistics.db")
cursor = conn.cursor()


###### Step 3 — Create tables

In [16]:
# Create assets table
cursor.execute("""
CREATE TABLE IF NOT EXISTS assets (
    asset_id INTEGER PRIMARY KEY,
    asset_code TEXT
);
""")

# Create locations table
cursor.execute("""
CREATE TABLE IF NOT EXISTS locations (
    location_id INTEGER PRIMARY KEY,
    latitude REAL,
    longitude REAL
);
""")

# Create shipments table (core transactional data)
cursor.execute("""
CREATE TABLE IF NOT EXISTS shipments (
    shipment_id INTEGER PRIMARY KEY,
    timestamp TEXT,
    asset_id INTEGER,
    location_id INTEGER,
    inventory_level REAL,
    shipment_status TEXT,
    temperature REAL,
    humidity REAL,
    traffic_status TEXT,
    waiting_time REAL,
    user_transaction_amount REAL,
    user_purchase_frequency REAL,
    logistics_delay_reason TEXT,
    asset_utilization REAL,
    demand_forecast REAL,
    logistics_delay INTEGER,
    FOREIGN KEY (asset_id) REFERENCES assets(asset_id),
    FOREIGN KEY (location_id) REFERENCES locations(location_id)
);
""")


###### Step 4 — Insert data into tables

In [17]:
# Unique suppliers from Asset_ID
suppliers = pd.DataFrame(df['Asset_ID'].unique(), columns=['supplier_name'])
suppliers['supplier_id'] = suppliers.index + 1
suppliers.to_sql('suppliers', conn, if_exists='append', index=False)

# Unique locations from Latitude + Longitude
locs = df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
locs['location_id'] = locs.index + 1
locs['location_name'] = locs.apply(lambda x: f"{x['Latitude']}, {x['Longitude']}", axis=1)
locs[['location_id', 'location_name']].to_sql('locations', conn, if_exists='append', index=False)

# Map suppliers & locations
df = df.merge(suppliers, left_on='Asset_ID', right_on='supplier_name')
df = df.merge(locs[['location_id', 'Latitude', 'Longitude']], on=['Latitude', 'Longitude'])

# Prepare shipments table
shipments_df = pd.DataFrame({
    'supplier_id': df['supplier_id'],
    'origin_id': df['location_id'],  # Assuming each record's location is origin
    'destination_id': df['location_id'],  # If you have a separate destination, replace this
    'ship_date': df['Timestamp'],
    'arrival_date': None,  # If not present in dataset
    'delayed': df['Logistics_Delay'],
    'route': df['Traffic_Status']
})
shipments_df.to_sql('shipments', conn, if_exists='append', index=False)


1000

###### Step 5 — Write analysis queries

###### 1. Delay rate by supplier

In [18]:
query1 = """
SELECT s.supplier_name, 
       ROUND(AVG(sh.delayed)*100, 2) AS delay_rate_pct, 
       COUNT(*) AS total_shipments
FROM shipments sh
JOIN suppliers s ON sh.supplier_id = s.supplier_id
GROUP BY s.supplier_name
ORDER BY delay_rate_pct DESC;
"""
pd.read_sql(query1, conn)


,supplier_name,delay_rate_pct,total_shipments
0,Truck_10,64.76,105
1,Truck_3,62.37,93
2,Truck_4,58.88,107
3,Truck_7,58.82,102
4,Truck_8,56.88,109
5,Truck_9,56.38,94
6,Truck_2,53.33,105
7,Truck_6,52.43,103
8,Truck_1,51.69,89
9,Truck_5,49.46,93


###### 2. Delay rate by origin location

In [19]:
query2 = """
SELECT l.location_name AS origin, 
       ROUND(AVG(sh.delayed) * 100, 2) AS delay_rate_pct
FROM shipments sh
JOIN locations l 
    ON sh.origin_id = l.location_id
GROUP BY l.location_name
ORDER BY delay_rate_pct DESC;
"""
pd.read_sql(query2, conn)


,origin,delay_rate_pct
0,"9.8934, 13.9028",100.0
1,"9.6491, 47.3499",100.0
2,"9.3294, -71.5299",100.0
3,"89.8701, 73.6867",100.0
4,"89.2765, 166.6548",100.0
...,...,...
995,"-10.0451, 79.8956",0.0
996,"-1.6787, 3.2441",0.0
997,"-0.5646, 175.2938",0.0
998,"-0.5355, 120.9793",0.0


###### 3. Delay rate by month

In [20]:
query3 = """
SELECT strftime('%Y-%m', ship_date) AS month,
       ROUND(AVG(CAST(delayed AS FLOAT)) * 100, 2) AS delay_rate_pct
FROM shipments
GROUP BY month
ORDER BY month;
"""
pd.read_sql(query3, conn)


,month,delay_rate_pct
0,2024-01,52.17
1,2024-02,61.04
2,2024-03,56.38
3,2024-04,56.98
4,2024-05,52.70
5,2024-06,66.23
6,2024-07,62.79
7,2024-08,58.44
8,2024-09,48.19
9,2024-10,53.01


###### 4. Top 5 routes by number of delays

In [23]:
query4 = """
SELECT 
    route, 
    COUNT(*) AS delayed_shipments,
    (SELECT COUNT(*) FROM shipments s2 WHERE s2.route = s1.route) AS total_shipments
FROM shipments s1
WHERE delayed = 1
GROUP BY route
ORDER BY delayed_shipments DESC
LIMIT 5;
"""
pd.read_sql(query4, conn)


,route,delayed_shipments,total_shipments
0,Heavy,327,327
1,Detour,124,345
2,Clear,115,328


###### 5. Supplier with most on-time shipments

In [24]:
query5 = """
SELECT 
    s.supplier_name, 
    COUNT(*) AS on_time_shipments
FROM shipments AS sh
JOIN suppliers AS s 
    ON sh.supplier_id = s.supplier_id
WHERE sh.delayed = 0
GROUP BY s.supplier_name
ORDER BY on_time_shipments DESC
LIMIT 1;
"""
pd.read_sql(query5, conn)


,supplier_name,on_time_shipments
0,Truck_6,49


###### 6. Average delay rate per route

In [25]:
query6 = """
SELECT 
    sh.route, 
    ROUND(AVG(sh.delayed) * 100, 2) AS delay_rate_pct
FROM shipments AS sh
GROUP BY sh.route
ORDER BY delay_rate_pct DESC;
"""
pd.read_sql(query6, conn)


,route,delay_rate_pct
0,Heavy,100.00
1,Detour,35.94
2,Clear,35.06


###### 7. Delay rate by origin-destination pair

In [29]:
query7 = """
SELECT 
    lo.location_name AS origin, 
    ld.location_name AS destination,
    ROUND(AVG(sh.delayed) * 100, 2) AS delay_rate_pct
FROM shipments AS sh
JOIN locations lo 
    ON sh.origin_id = lo.location_id
JOIN locations ld 
    ON sh.destination_id = ld.location_id
GROUP BY origin, destination
ORDER BY delay_rate_pct DESC
LIMIT 10;
"""

pd.read_sql(query7, conn)


,origin,destination,delay_rate_pct
0,"-0.1148, -127.6784","-0.1148, -127.6784",100.0
1,"-0.8539, -177.5496","-0.8539, -177.5496",100.0
2,"-0.9333, 124.8495","-0.9333, 124.8495",100.0
3,"-1.0776, 51.9075","-1.0776, 51.9075",100.0
4,"-1.28, 1.0168","-1.28, 1.0168",100.0
5,"-1.4258, 112.8174","-1.4258, 112.8174",100.0
6,"-1.8123, -156.207","-1.8123, -156.207",100.0
7,"-10.7414, -56.5849","-10.7414, -56.5849",100.0
8,"-10.8232, 144.0982","-10.8232, 144.0982",100.0
9,"-10.9298, -89.0306","-10.9298, -89.0306",100.0


In [30]:
# Run the query
df = pd.read_sql(query7, conn)

# Close the connection
conn.close()

# Display the result
df


,origin,destination,delay_rate_pct
0,"-0.1148, -127.6784","-0.1148, -127.6784",100.0
1,"-0.8539, -177.5496","-0.8539, -177.5496",100.0
2,"-0.9333, 124.8495","-0.9333, 124.8495",100.0
3,"-1.0776, 51.9075","-1.0776, 51.9075",100.0
4,"-1.28, 1.0168","-1.28, 1.0168",100.0
5,"-1.4258, 112.8174","-1.4258, 112.8174",100.0
6,"-1.8123, -156.207","-1.8123, -156.207",100.0
7,"-10.7414, -56.5849","-10.7414, -56.5849",100.0
8,"-10.8232, 144.0982","-10.8232, 144.0982",100.0
9,"-10.9298, -89.0306","-10.9298, -89.0306",100.0


###### Step 6 — Close DB connection

In [32]:
conn.close()